# The `super` Built-in Function: For Better or Worse?


So far, I’ve mentioned Python’s `super` built-in function only briefly in passing because
it is relatively uncommon and may even be controversial to use. Given this call’s increased 
visibility in recent years, though, it merits some further elaboration in this
edition. Besides introducing `super`, this section also serves as a language design case
study to close out a chapter on so many tools whose presence may to some seem curious
in a scripting language like Python.

Some of this section calls this proliferation of tools into question, and I encourage you
to judge any subjective content here for yourself (and we’ll return to such things at the
end of this book after we’ve expanded on other advanced tools such as metaclasses and
descriptors). Still, Python’s rapid growth rate in recent years represents a strategic 
decision point for its community going forward, and `super` seems as good a representative
example as any.

## The Great `super` Debate


As noted in [Chapter 28]() and [Chapter 29](), Python has a `super` built-in function that can
be used to invoke superclass methods generically, but was deferred until this point of
the book. This was deliberate—because `super` has substantial downsides in typical
code, and a sole use case that seems obscure and complex to many observers, most
beginners are better served by the traditional explicit-name call scheme used so far. See
[the sidebar “What About `super`?” on page 831]() in [Chapter 28]() for a brief summary of
the rationale for this policy.

The Python community itself seems split on this subject, with online articles about it
running the gamut from “Python’s Super Considered Harmful” to “Python’s `super()`
considered super!”  Frankly, in my live classes this call seems to be most often of interest
to Java programmers starting to use Python anew, because of its conceptual similarity
to a tool in that language (many a new Python feature ultimately owes its existence to
programmers of other languages bringing their old habits to a new model). Python’s
`super` is not Java’s—it translates differently to Python’s multiple inheritance, and has
a use case beyond Java’s—but it has managed to generate both controversy and misunderstanding 
since its conception.

This book postponed the `super` call until now (and omitted it almost entirely in prior
editions) because it has significant issues—it’s prohibitively cumbersome to use in 2.X,
differs in form between 2.X and 3.X, is based upon unusual semantics in 3.X, and mixes
poorly with Python’s multiple inheritance and operator overloading in typical Python
code. In fact, as we’ll see, in some code `super` can actually mask problems, and 
discourage a more explicit coding style that offers better control.

In its defense, this call does have a valid use case too—cooperative same-named method
dispatch in diamond multiple inheritance trees—but it seems to ask a lot of newcomers.
It requires that `super` be used universally and consistently (if not neurotically), much
like `__slots__` discussed earlier; relies on the arguably obscure MRO algorithm to order
calls; and addresses a use case that seems far more the exception than the norm in
Python programs. In this role, `super` seems an advanced tool based upon esoteric principles, 
which may be beyond much of Python’s audience, and seems artificial to real
program goals. That aside, its expectation of universal use seems unrealistic for the vast
amount of existing Python code.

Because of all these factors, this introductory-level book has preferred the traditional
explicit-name call scheme thus far and recommends the same for newcomers. You’re
better off learning the traditional scheme first, and might be better off sticking with
that in general, rather than using an extra special-case tool that may not work in some
contexts, and relies on arcane magic in the valid but atypical use case it addresses. This
is not just your author’s opinion; despite its advocate’s best intentions, `super` is not
widely recognized as “best practice” in Python today, for completely valid reasons.

On the other hand, just as for other tools the increasing use of this call in Python code
in recent years makes it no longer optional for many Python programmers—the first
time you see it, it’s officially mandatory! For readers who may wish to experiment with
`super`, and for other readers who may have it imposed upon them, this section provides
a brief look at this tool and its rationale—beginning with alternatives to it.

## Traditional Superclass Call Form: Portable, General


In general, this book’s examples prefer to call back to superclass methods when needed
by naming the superclass explicitly, because this technique is traditional in Python,
because it works the same in both Python 2.X and 3.X, and because it sidesteps limitations 
and complexities related to this call in both 2.X and 3.X. As shown earlier, the
traditional superclass method call scheme to augment a superclass method works as
follows:


In [ ]:
class C:                    # In Python 2.X and 3.X
    def act(self):
        print('spam')

class D(C):
    def act(self):
        C.act(self)         # Name superclass explicitly, pass self
        print('eggs')

X = D()
X.act()

spam
eggs


This form works the same in 2.X and 3.X, follows Python’s normal method call mapping model, 
applies to all inheritance tree forms, and does not lead to confusing behavior 
when operator overloading is used. To see why these distinctions matter, let’s
see how `super` compares.

## Basic `super` Usage and Its Tradeoffs


In this section, we’ll both introduce `super` in basic, *single-inheritance mode*, and look
at its perceived downsides in this role. As we’ll find, in this context `super` does work as
advertised, but is not much different from traditional calls, relies on unusual semantics,
and is cumbersome to deploy in 2.X. More critically, as soon as your classes grow to
use multiple inheritance, this super usage mode can both mask problems in your code
and route calls in ways you may not expect.

### Odd semantics: A magic proxy in Python 3.X


The `super` built-in actually has two intended roles. The more esoteric of these—
cooperative multiple inheritance dispatch protocols in diamond multiple-inheritance trees
(yes, a mouthful!)—relies on the 3.X MRO, was borrowed from the Dylan language,
and will be covered later in this section.

The role we’re interested in here is more commonly used, and more frequently requested 
by people with Java backgrounds—to allow superclasses to be named *generically*
in inheritance trees. This is intended to promote simpler code maintenance, and to
avoid having to type long superclass reference paths in calls. In Python 3.X, this call
seems at least at first glance to achieve this purpose well:

In [ ]:
class C:  # In Python 3.X (only: see 2.X super form ahead)
    def act(self):
        print('spam')

class D(C):
    def act(self):
        super().act() # Reference superclass generically, omit self
        print('eggs')

X = D()
X.act()

spam
eggs


This works, and minimizes code changes—you don’t need to update the call if `D`’s
superclass changes in the future. One of the biggest downsides of this call in 3.X,
though, is its *reliance on deep magic*: though prone to change, it operates today by
inspecting the call stack in order to automatically locate the `self` argument and find
the superclass, and pairs the two in a special *proxy object* that routes the later call to
the superclass version of the method. If that sounds complicated and strange, it’s because 
it is. In fact, this call form doesn’t work at all outside the context of a class’s
method:

In [ ]:
super               # A "magic" proxy object that routes later calls

super

In [ ]:
super()

RuntimeError: super(): no arguments

In [ ]:
class E(C):
    def method(self):           # self is implicit in super...only!
        proxy = super()         # This form has no meaning outside a method
        print(proxy)            # Show the normally hidden proxy object
        proxy.act()             # No arguments: implicitly calls superclass method!

E().method()

<super: <class 'E'>, <E object>>
spam


Really, this call’s semantics resembles nothing else in Python—it’s neither a bound nor
unbound method, and somehow finds a `self` even though you omit one in the call. In
single inheritance trees, a superclass is available from `self` via the path
`self.__class__.__bases__[0]`, but the heavily implicit nature of this call makes this
difficult to see, and even flies in the face of Python’s explicit `self` policy that holds true
*everywhere else*. That is, this call violates a fundamental Python idiom for a single use
case. It also soundly contradicts Python’s longstanding EIBTI design rule (run an 
“import this” for more on this rule).

### Pitfall: Adding multiple inheritance naively


Besides its unusual semantics, even in 3.X this `super` role applies most directly to single
inheritance trees, and can become problematic as soon as classes employ multiple inheritance 
with traditionally coded classes. This seems a major limitation of scope; due
to the utility of *mix-in* classes in Python, multiple inheritance from disjoint and 
independent superclasses is probably more the norm than the exception in realistic code.
The `super` call seems a recipe for disaster in classes coded to naively use its basic mode,
without allowing for its much more subtle implications in multiple inheritance trees.

The following illustrates the trap. This code begins its life happily deploying `super` in
single-inheritance mode to invoke a method one level up from C:

In [ ]:
class A:                        # In Python 3.X
    def act(self): print('A')

class B:
    def act(self): print('B')

class C(A):
    def act(self):
        super().act()           # super applied to a single-inheritance tree

X = C()
X.act()

A


If such classes later grow to use more than one superclass, though, `super` can become
error-prone, and even unusable—it does not raise an exception for multiple inheritance
trees, but will naively pick just the *leftmost* superclass having the method being run
(technically, the first per the MRO), which may or may not be the one that you want:

In [ ]:
class C(A, B):              # Add a B mix-in class with the same method
    def act(self):
        super().act()       # Doesn't fail on multi-inher, but picks just one!

X = C()
X.act()

A


In [ ]:
class C(B, A):
    def act(self):
        super().act()       # If B is listed first, A.act() is no longer run!

X = C()
X.act()

B


Perhaps worse, this *silently masks* the fact that you should probably be selecting 
superclasses *explicitly* in this case, as we learned earlier in both [this chapter]() and [its predecessor](). In other words, `super` usage may obscure a common source of errors in Python
—one so common that it shows up again in [this part’s “Gotchas.”]() If you may need to
use direct calls later, why not use them earlier too?

In [ ]:
class C(A, B):              # Traditional form
    def act(self):          # You probably need to be more explicit here
        A.act(self)         # This form handles both single and multiple inher
        B.act(self)         # And works the same in both Python 3.X and 2.X

X = C()                     # So why use the super() special case at all?
X.act()

A
B


As we’ll see in a few moments, you might also be able to address such cases by deploying
`super` calls in every class of the tree. But that’s also one of the biggest downsides of
`super`—why code it in every class, when it’s usually not needed, and when using the
preceding simpler traditional form in a single class will usually suffice? Especially in
existing code—and new code that uses existing code—this `super` requirement seems
harsh, if not unrealistic.

Much more subtly, as we’ll also see ahead, once you step up to multiple inheritance
calls this way, the `super` calls in your code might not invoke the class you expect them
to. They’ll be routed per the MRO order, which, depending on where else `super` might
be used, may invoke a method in a class that is *not the caller’s superclass at all*—an
implicit ordering that might make for interesting debugging sessions! Unless you 
completely understand what `super` means once multiple inheritance is introduced, you may
be better off not deploying it in single-inheritance mode either.

This coding situation isn’t nearly as abstract as it may seem. Here’s a real-world example
of such a case, taken from the *PyMailGUI* case study in [Programming Python]()—the
following very typical Python classes use multiple inheritance to mix in both application
logic and window tools from independent, standalone classes, and hence must invoke
*both* superclass constructors explicitly with direct calls by name. As coded, a
`super().__init__()` here would run only one constructor, and adding `super` throughout
this example’s disjoint class trees would be more work, would be no simpler, and
wouldn’t make sense in tools meant for arbitrary deployment in clients that may use
`super` or not:

```python
    class PyMailServerWindow(PyMailServer, windows.MainWindow):
        "a Tk, with extra protocol and mixed-in methods"
        def __init__(self):
            windows.MainWindow.__init__(self, appname, srvrname)
            PyMailServer.__init__(self)

    class PyMailFileWindow(PyMailFile, windows.PopupWindow):
        "a Toplevel, with extra protocol and mixed-in methods"
        def __init__(self, filename):
            windows.PopupWindow.__init__(self, appname, filename)
            PyMailFile.__init__(self, filename)
```

The crucial point here is that using `super` for just the single inheritance cases 
where it applies most clearly is a potential source of error and confusion, and means that programmers must remember two ways to accomplish the same goal, when just one—
explicit direct calls—could suffice for all cases.

In other words, unless you can be sure that you will never add a second superclass to
a class in a tree over your software’s entire lifespan, you cannot use `super` in 
single-inheritance mode without understanding and allowing for its much more sophisticated
role in multiple-inheritance trees. We’ll discuss the latter ahead, but it’s not optional
if you deploy `super` at all.

From a more practical view, it’s also not clear that the trivial amount of *code maintenance* that this `super` role is envisioned to avoid fully justifies its presence. In Python
practice, superclass names in headers are rarely changed; when they are, there are usually 
at most a very small number of superclass calls to update within the class. And
consider this: if you add a new superclass in the future that doesn’t use `super` (as in the
preceding example), you’ll have to either wrap it in an adaptor proxy or augment all
the `super` calls in your class to use the traditional explicit-name call scheme anyhow—
a maintenance task that seems just as likely, but perhaps more error-prone if you’ve
grown to rely on `super` magic.

### Limitation: Operator overloading


As briefly noted in Python’s library manual, `super` also doesn’t fully work in the 
presence of `__X__` operator overloading methods. If you study the following code, you’ll see
that direct named calls to overload methods in the superclass operate normally, but
using the `super` result in an expression fails to dispatch to the superclass’s overload
method:

In [ ]:
class C:                            # In Python 3.X
    def __getitem__(self, ix):      # Indexing overload method
        print('C index')

class D(C):
    def __getitem__(self, ix):      # Redefine to extend here
        print('D index')
        C.__getitem__(self, ix)     # Traditional call form works
        super().__getitem__(ix)     # Direct name calls work too
        super()[ix]                 # But operators do not! (__getattribute__)

X = C()
X[99]

C index


In [ ]:
X = D()
X[99]

D index
C index
C index


TypeError: 'super' object is not subscriptable

This behavior is due to the very same new-style (and 3.X) class change described earlier
in [this chapter]() (see [“Attribute Fetch for Built-ins Skips Instances” on page 987]())—
because the proxy object returned by `super` uses `__getattribute__` to catch and dispatch
later method calls, it fails to intercept the automatic `__X__` method invocations run by
built-in operations including expressions, as these begin their search in the class instead
of the instance. This may seem less severe than the multiple-inheritance limitation, but
operators should generally work the same as the equivalent method call, especially for
a built-in like this. Not supporting this adds another exception for `super` users 
to confront and remember.

Other languages’ mileage may vary, but in Python, `self` is explicit, multiple-inheritance
mix-ins and operator overloading are common, and superclass name updates are rare.
Because `super` adds an odd special case to the language—one with strange semantics,
limited scope, rigid requirements, and questionable reward—most Python programmers 
may be better served by the more broadly applicable traditional call scheme. While
`super` has some advanced applications too that we’ll study ahead, they may be too
obscure to warrant making it a mandatory part of every Python programmer’s toolbox.

So why use a technique that works in only limited contexts instead of one that works
in many more? Though its basis is complex, the next sections attempt to rally support
for the `super` cause.

## The `super` Upsides: Tree Changes and Dispatch


Having just shown you the downsides of `super`, I should also confess that I’ve been
tempted to use this call in code that would only ever run on 3.X, and which used a very
long superclass reference path through a module package (that is, mostly for laziness,
but coding brevity can matter too). To be fair, `super` may still be useful in some use
cases, the chief among which merit a brief introduction here:

  + *Changing class trees at runtime*: When a superclass may be changed at runtime, it’s
    not possible to hardcode its name in a call expression, but it is possible to dispatch
    calls via `super`.  
    On the other hand, this case is extremely rare in Python programming, and other
    techniques can often be used in this context as well.

  + *Cooperative multiple inheritance method dispatch*: When multiple inheritance trees
    must dispatch to the same-named method in multiple classes, `super` can provide a
    protocol for orderly call routing.  
    On the other hand, the class tree must rely upon the ordering of classes by the
    MRO—a complex tool in its own right that is artificial to the problem a program
    is meant to address—and must be coded or augmented to use super in each version
    of the method in the tree to be effective. Such dispatch can also often be implemented 
    in other ways (e.g., via instance state).

As discussed earlier, `super` can also be used to select a superclass generically as long as
the MRO’s default makes sense, though in traditional code naming a superclass explicitly 
is often preferable, and may even be required. Moreover, even valid `super` use
cases tend to be uncommon in many Python programs—to the point of seeming academic 
curiosity to some. The two cases just listed, however, are most often cited as
`super` rationales, so let’s take a quick look at each.

### Runtime Class Changes and `super`


Superclass that might be changed at runtime dynamically preclude hardcoding their
names in a subclass’s methods, while `super` will happily look up the current superclass
dynamically. Still, this case may be too rare in practice to warrant the super model by
itself, and can often be implemented in other ways in the exceptional cases where it is
needed. To illustrate, the following changes the superclass of `C` dynamically by changing
the subclass’s `__bases__` tuple in 3.X:


In [ ]:
class X:
    def m(self): print('X.m')

class Y:
    def m(self): print('Y.m')

class C(X):                         # Start out inheriting from X
    def m(self): super().m()        # Can't hardcode class name here

i = C()
i.m()

X.m


In [ ]:
C.__bases__ = (Y,)                  # Change superclass at runtime!
i.m()

Y.m


This works (and shares behavior-morphing goals with other deep magic, such as
changing an instance’s `__class__`), but seems rare in the extreme. Moreover, there may
be other ways to achieve the same effect—perhaps most simply, calling through the
current superclass tuple’s value indirectly: special code to be sure, but only for a very
special case (and perhaps not any more special than implicit routing by MROs):


In [ ]:
class C(X):
    def m(self): C.__bases__[0].m(self)     # Special code for a special case

i = C()
i.m()

X.m


In [ ]:
C.__bases__ = (Y,)                          # Same effect, without super()
i.m()

Y.m


Given the preexisting alternatives, this case alone doesn’t seem to justify super, though
in more complex trees, the next rationale—based on the tree’s MRO order instead of
physical superclass links—may apply here as well.

### Cooperative Multiple Inheritance Method Dispatch


The second of the use cases listed earlier is the main rationale commonly given for
`super`, and also borrows from other programming languages (most notably, Dylan),
where its use case may be more common than it is in typical Python code. It generally
applies to diamond pattern multiple inheritance trees, discussed earlier in [this chapter](),
and allows for cooperative and conformant classes to route calls to a *same-named method* 
coherently among multiple class implementations. Especially for constructors,
which have multiple implementations normally, this can simplify call routing protocol
when used consistently.

In this mode, each `super` call selects the method from a *next class* following it in the
MRO ordering of the class of the `self` subject of a method call. The MRO was introduced 
earlier; it’s the path Python follows for inheritance in new-style classes. Because
the MRO’s linear ordering depends on which class `self` was made from, the order of
method dispatch orchestrated by `super` can vary per class tree, and visits each class just
once as long as all classes use `super` to dispatch.

Since every class participates in a diamond under object in 3.X (and 2.X new-style
classes), the applications are broader than you might expect. In fact, some of the earlier
examples that demonstrated `super` shortcomings in multiple inheritance trees could
use this call to achieve their dispatch goals. To do so, however, `super` must be used
*universally* in the class tree to ensure that method call chains are passed on—a fairly
major requirement that may be difficult to enforce in much existing and new code.

#### The basics: Cooperative `super` call in action


Let’s take a look at what this role means in code. In this and the following sections,
we’ll both learn how `super` works, and explore the tradeoffs it implies along the way.
To get started, consider the following *traditionally* coded Python classes (condensed
somewhat here as usual for space):

In [ ]:
class B:
    def __init__(self): print('B.__init__')     # Disjoint class tree branches

class C:
    def __init__(self): print('C.__init__')

class D(B, C): pass

x = D()                                         # Runs leftmost only by default

B.__init__


In this case, superclass tree branches are *disjoint* (they don’t share a common explicit
ancestor), so subclasses that combine them must call through each superclass by name
—a common situation in much existing Python code that `super` cannot address directly
without code changes:

In [ ]:
class D(B, C):
 def __init__(self):                            # Traditional form
    B.__init__(self)                            # Invoke supers by name
    C.__init__(self)

x = D()

B.__init__
C.__init__


In *diamond* class tree patterns, though, *explicit-name* calls may by default trigger the
top-level class’s method more than once, though this might be subverted with additional 
protocols (e.g., status markers in the instance):

In [ ]:
class A:
    def __init__(self): print('A.__init__')

class B(A):
    def __init__(self): print('B.__init__'); A.__init__(self)

class C(A):
    def __init__(self): print('C.__init__'); A.__init__(self)

x = B()

B.__init__
A.__init__


In [ ]:
x = C()

C.__init__
A.__init__


In [ ]:
class D(B, C): pass                         # Still runs leftmost only

x = D()

B.__init__
A.__init__


In [ ]:
class D(B, C):
    def __init__(self):                     # Traditional form
        B.__init__(self)                    # Invoke both supers by name
        C.__init__(self)

x = D()                                     # But this now invokes A twice!

B.__init__
A.__init__
C.__init__
A.__init__


By contrast, if all classes use `super`, or are appropriately coerced by proxies to behave
as if they do, the method calls are dispatched according to class order in the MRO, such
that the top-level class’s method is run just once:

In [ ]:
class A:
    def __init__(self): print('A.__init__')

class B(A):
    def __init__(self): print('B.__init__'); super().__init__()

class C(A):
    def __init__(self): print('C.__init__'); super().__init__()

x = B()                     # Runs B.__init__, A is next super in self's B MRO

B.__init__
A.__init__


In [ ]:
x = C()

C.__init__
A.__init__


In [ ]:
class D(B, C): pass
x = D()                     # Runs B.__init__, C is next super in self's D MRO!

B.__init__
C.__init__
A.__init__


The real magic behind this is the linear MRO list constructed for the class of `self`—
because each class appears just once on this list, and because `super` dispatches to the
*next* class on this list, it ensures an orderly invocation chain that visits each class just
once. Crucially, the *next* class following `B` in the MRO differs depending on the class
of `self`—it’s `A` for a `B` instance, but `C` for a `D` instance, accounting for the order of 
constructors run:

In [ ]:
B.__mro__

(__main__.B, __main__.A, object)

In [ ]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

The MRO and its algorithm were presented earlier in [this chapter](). By selecting a next
class in the MRO sequence, a `super` call in a class’s method *propagates* the call through
the tree, so long as all classes do the same. In this mode `super` does not necessarily
choose a superclass at all; it picks the next in the linearized MRO, which might be a
*sibling*—or even a *lower* relative—in the class tree of a given instance. See [“Tracing the
MRO” on page 1002]() for other examples of the path `super` dispatch would follow, 
especially for nondiamonds.

The preceding works—and may even seem clever at first glance—but its scope may
also appear limited to some. Most Python programs do not rely on the nuances of
diamond pattern multiple inheritance trees (in fact, many Python programmers I’ve
met do not know what the term means!). Moreover, `super` applies most directly to single
inheritance and cooperative diamond cases, and may seem superfluous for disjoint
nondiamond cases, where we might want to invoke superclass methods selectively or
independently. Even cooperative diamonds can be managed in other ways that may
afford programmers more control than an automatic MRO ordering can. To evaluate
this tool objectively, though, we need to look deeper.

#### Constraint: Call chain anchor requirement


The `super` call comes with complexities that may not be apparent on first encounter,
and may even seem initially like features. For example, because *all* classes inherit from
`object` in 3.X automatically (and explicitly in 2.X new-style classes), the MRO ordering
can be used even in cases where the diamond is only implicit—in the following, triggering 
constructors in independent classes automatically: